In [ ]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

from preprocessing import limpiar_datos, aplicar_dummy_variables_encoding, eliminar_features, entrenar_iterative_imputer, imputar_missings_iterative
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc,graficar_matriz_confusion
from funciones_auxiliares import traer_datasets, traer_dataset_prediccion_final, separar_dataset, encontrar_hiperparametros_RGSCV, mapear_target_binario

# Arbol de clasificación

### Obtención de los datos

In [ ]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)

X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_test.is_copy=False

y_train.set_index('id', inplace=True)
y_train = y_train.sort_values(by=['id'], ascending=True).copy()

y_test.set_index('id', inplace=True)
y_test = y_test.sort_values(by=['id'], ascending=True).copy()


ESTO DEBERÍA IR EN preprocessing.py !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! :

In [ ]:
features_continuas = ['id','horas_de_sol','humedad_tarde', 'humedad_temprano', 'mm_evaporados_agua', 'mm_lluvia_dia', 'nubosidad_tarde', 'nubosidad_temprano', 'presion_atmosferica_tarde', 'presion_atmosferica_temprano', 'rafaga_viento_max_velocidad','temp_max', 'temp_min', 'temperatura_tarde', 'temperatura_temprano',  'velocidad_viendo_tarde','velocidad_viendo_temprano']

### Definiendo distintos preprocesamientos

Definiremos entonces dos preprocesamientos distintos a comparar para este modelo, y quedarnos con el mejor de ellos cuando probemos en holdout.

In [ ]:
def preprocesamiento_1(df_original:pd.DataFrame, imputer=None):
    df = df_original.copy(deep=True)
    df = limpiar_datos(df)
    df = aplicar_dummy_variables_encoding(df, ['llovieron_hamburguesas_hoy'])
    eliminar_features(df, ['dia','barrio', 'direccion_viento_tarde', 'direccion_viento_temprano', 'rafaga_viento_max_direccion'])
    
    if(imputer is None):
        imputer = entrenar_iterative_imputer(df)
    df = imputar_missings_iterative(df, imputer)
    
    return df, imputer

Vemos que el primero de ellos primero corrije los datos con formato erróneo del dataset, como por ejemplo direcciones IP registradas en la feature 'presion_atmosférica_tarde', o datos que deberían ser NaNs pero son whitespace. Luego se aplica dummy encoding a la única feature categórica que nos resultó importante del análisis en el TP1. Pasamos a eliminar el resto de features categóricas y finalmente completamos los missings con el IterativeImputer.

In [ ]:
def preprocesamiento_2(df_original:pd.DataFrame, imputer=None, seleccion_random_forest=None):
    df = df_original.copy(deep=True)
    df = limpiar_datos(df)
    df = aplicar_dummy_variables_encoding(df, ['llovieron_hamburguesas_hoy', 'dia','barrio', 'direccion_viento_tarde', 'direccion_viento_temprano', 'rafaga_viento_max_direccion'])
    
    if(imputer == None):
        imputer = entrenar_iterative_imputer(df)
    df = imputar_missings_iterative(df, imputer)
    
    if(seleccion_random_forest is None):
        sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
        sel.fit(df, y_train.to_numpy().ravel())
        seleccion_random_forest= df.columns[(sel.get_support())]
    
    eliminar_features(df, df.columns.difference(seleccion_random_forest))
    
    return df, imputer, seleccion_random_forest

El segundo preprocesamiento primero corrije los datos con formato erróneo del dataset, como por ejemplo direcciones IP registradas en la feature 'presion_atmosférica_tarde', o datos que deberían ser NaNs pero son whitespace. Luego se aplica dummy encoding a todas las features categóricas. Una vez hecho esto, completamos los missings con el IterativeImputer para poder utilizar un clasificador Random Forest como método de selección de atributos. Gracias a esta selección, podemos droppear del dataframes aquellas features que RandomForest no haya considerado importante para terminar el preprocesamiento.

### Aplicamos preprocesamientos 1 y 2 para obtener X_train_1 / X_test_1 y X_train_2 / X_test_2 respectivamente

In [ ]:
X_train_1, imputer_entrenado_1 = preprocesamiento_1(X_train)
X_test_1, imputer_entrenado_1 = preprocesamiento_1(X_test, imputer_entrenado_1)

In [ ]:
X_train_2, imputer_entrenado_2, seleccion_RF = preprocesamiento_2(X_train)
X_test_2, imputer_entrenado_2, seleccion_RF = preprocesamiento_2(X_test, imputer_entrenado_2, seleccion_RF)

### Búsqueda de Hiperparámetros para el modelo que entrenará sobre X_train_1

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento.
Estos parámetros serán:
* criterion: La función que evalua la calidad de una división en el árbol
* min_samples_leaf: La mínima cantidad de muestras requeridas para dividir un nodo
* max_depth: La máxima profundiad que le permitimos al árbol. Un árbol muy profundo implicaría un overfit.

In [ ]:
params = {'criterion': ['gini','entropy'], 'min_samples_leaf':np.arange(1,16),'max_depth': np.arange(1,30)}
hiperparametros = encontrar_hiperparametros_RGSCV(DecisionTreeClassifier(), params=params, x_np=X_train_1.to_numpy(), y_np=y_train.to_numpy())

In [ ]:
criterio_elegido_1 = hiperparametros['criterion']
profundidad_elegida_1 = hiperparametros['max_depth']
min_leaf_elegido_1 = hiperparametros['min_samples_leaf']
print(f'Mejor criterio: {criterio_elegido_1}')
print(f'Mejor profundidad máxima: {profundidad_elegida_1}')
print(f'Mejor mínima cantidad de instancias por hoja: {min_leaf_elegido_1}')

### Entrenando el Modelo 1 sobre X_train_1 con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train_1, y_train)):
    arbol_clasificacion_1 = DecisionTreeClassifier(max_depth=profundidad_elegida_1, min_samples_leaf=min_leaf_elegido_1, criterion=criterio_elegido_1, random_state=117)
    arbol_clasificacion_1.fit(X_train_1.iloc[train_index], y_train.iloc[train_index])
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], arbol_clasificacion_1.predict(X_train_1.iloc[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

### Búsqueda de Hiperparámetros para el modelo que entrenará sobre X_train_2

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento.
Estos parámetros serán:
* criterion: La función que evalua la calidad de una división en el árbol
* min_samples_leaf: La mínima cantidad de muestras requeridas para dividir un nodo
* max_depth: La máxima profundiad que le permitimos al árbol. Un árbol muy profundo implicaría un overfit.

In [ ]:
params = {'criterion': ['gini','entropy'], 'min_samples_leaf':np.arange(1,16),'max_depth': np.arange(1,30)}
hiperparametros = encontrar_hiperparametros_RGSCV(DecisionTreeClassifier(), params=params, x_np=X_train_2.to_numpy(), y_np=y_train.to_numpy())

In [ ]:
criterio_elegido_2 = hiperparametros['criterion']
profundidad_elegida_2 = hiperparametros['max_depth']
min_leaf_elegido_2 = hiperparametros['min_samples_leaf']
print(f'Mejor criterio: {criterio_elegido_2}')
print(f'Mejor profundidad máxima: {profundidad_elegida_2}')
print(f'Mejor mínima cantidad de instancias por hoja: {min_leaf_elegido_2}')

### Entrenando el Modelo 2 sobre X_train_2 con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train_2, y_train)):
    arbol_clasificacion_2 = DecisionTreeClassifier(max_depth=profundidad_elegida_2, min_samples_leaf=min_leaf_elegido_2, criterion=criterio_elegido_2, random_state=117)
    arbol_clasificacion_2.fit(X_train_2.iloc[train_index], y_train.iloc[train_index])
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], arbol_clasificacion_2.predict(X_train_2.iloc[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

### Predicción del modelo sobre holdout con el modelo que mejor resultó al hacer CrossValidation: Modelo 1

Una vez entrenado y validado el modelo con CrossValidation, viendo que las métricas resultantes fueron razonables y no dan indicio de overfit o mala división del dataset, pasaremos a probar el modelo en la partición Holdout. Para esto usaremos los mejores hiperparámetros hallados previamente en Training.

In [ ]:
arbol_holdout = DecisionTreeClassifier(max_depth=profundidad_elegida_1, min_samples_leaf=min_leaf_elegido_1, criterion=criterio_elegido_1, random_state=117)
arbol_holdout.fit(X_train_1.to_numpy(), y_train.to_numpy())

In [ ]:
mapeo_binario_v = np.vectorize(mapear_target_binario)
y_pred = arbol_holdout.predict(X_test_1.to_numpy())
y_pred_binario = mapeo_binario_v(y_pred)
y_pred_proba = arbol_holdout.predict_proba(X_test_1.to_numpy())[:, 1]
y_test_binario = y_test['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0}).to_numpy()
print(classification_report(y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy(), y_pred))

### Curva AUC ROC


In [ ]:
graficar_auc_roc(y_test_binario, y_pred_proba)

### Matriz de confusion

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred_binario)

## Predicción con el dataset nuevo
A continuación, realizamos la predicción con el árbol sobre el dataset de predicciones nuevo, y escribimos los resultados al archivo 'ArbolDeDecision.csv'

In [ ]:
from funciones_auxiliares import exportar_prediccion_final

df_prediccion_final = traer_dataset_prediccion_final()
ids = df_prediccion_final['id'].to_numpy()

df_prediccion_final, imputer_entrenado_1 = preprocesamiento_1(df_prediccion_final, imputer_entrenado_1)
predicciones = arbol_holdout.predict(df_prediccion_final.to_numpy())

exportar_prediccion_final(ids, predicciones, 'arbol_de_decision')